This notebook reads xls population tables published by the UK Office of National Statistics and produces an xarray Dataset to be read by the AgriFoodPy package when importing the population modules.

``` python
from agrifoodpy.population.population import UK_ONS
```

The dimension coordinates of the array are
- **Year**
- **Datatype**

Where the datatypy dimension has the following coordinates:
- Females
- Males
- Total

The original xls files read in this notebook can be obtained here:

The ONS produces estimates based on different fertility, mortality and migration assumptions which can be downloaded in zip format here:
https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationprojections/datasets/z1zippedpopulationprojectionsdatafilesuk/2022based/uk.zip

- Principal	(ppp)
- High fertility	(hpp)
- Low fertility	(lpp)
- High life expectancy	(php)
- Low life expectancy	(plp)
- High migration	(pph)
- Low migration	(ppl)
- High population	(hhh)
- Low population	(lll)
- Zero net migration	(ppz)
- Young age structure	(hlh)
- Old age structure	(lhl)
- Replacement fertility	(rpp)
- Migration cateogry variant	(ppu)
- No long-term mortality improvement	(pnp)

We create a DataArray for each of these projections and concatenate them into a single Dataset.

In [1]:
import pandas as pd
import xarray as xr
import numpy as np

In [2]:
# Dictionary to map the project codes to their names
proj_dict = {
"ppp": "Principal",
"hpp": "High fertility",
"lpp": "Low fertility",
"php": "High life expectancy",
"plp": "Low life expectancy",
"pph": "High migration",
"ppl": "Low migration",
"hhh": "High population",
"lll": "Low population",
"ppz": "Zero net migration",
"hlh": "Young age structure",
"lhl": "Old age structure",
"rpp": "Replacement fertility",
"ppu": "Migration cateogry variant",
"pnp": "No long-term mortality improvement",
}


In [3]:
def read_proj_data(proj_key):
    """
    Reads population data from ONS and returns a DataArray with the population data.
    The obtained pandas dataframe contains the following columns:
    - Sex: Male, Female, Total
    - Age: 0, 1, 2, 3,  ..., 105-109, 110+
    - 2022, 2012, ..., 2122
    """

    # Read data
    fname = f"../../data/population/ONS_population/uk_{proj_key}_machine_readable.xlsx"
    df = pd.read_excel(fname, sheet_name="Population")

    # Group by Sex and sum over Age for each year
    pop_by_sex = df.groupby('Sex').sum(numeric_only=True).reset_index()

    # Add Total row
    pop_by_sex.loc[2] = pop_by_sex.loc[0] + pop_by_sex.loc[1]
    pop_by_sex.loc[2, 'Sex'] = 'Total'
    pop_by_sex

    # Get year columns
    year_cols = [col for col in pop_by_sex.columns[1:]]
    years = [int(year) for year in year_cols]

    # Create DataArray
    pop_array = xr.DataArray(
        data=pop_by_sex[year_cols].values,
        dims=['Datatype', 'Year'],
        coords={
            'Datatype': pop_by_sex['Sex'].values,
            'Year': years
        },
        name=proj_dict[proj_key]
    
    )

    return pop_array


In [4]:
# Create list of data arrays
dataArrays = [read_proj_data(key) for key in proj_dict.keys()]

In [5]:
# Create empty dataset and populate with data
ONS_population_dataset = xr.Dataset(
    coords={
            'Datatype': ["Females", "Males", "Total"],
            'Year': np.arange(2022, 2122+1)
        },
)

for da in dataArrays:
    ONS_population_dataset[da.name] = da

In [6]:
# Print data
ONS_population_dataset

<xarray.Dataset>
Dimensions:                             (Datatype: 3, Year: 101)
Coordinates:
  * Datatype                            (Datatype) object 'Females' ... 'Total'
  * Year                                (Year) int64 2022 2023 ... 2121 2122
Data variables: (12/15)
    Principal                           (Datatype, Year) int64 34494935 ... 8...
    High fertility                      (Datatype, Year) int64 34494935 ... 9...
    Low fertility                       (Datatype, Year) int64 34494935 ... 6...
    High life expectancy                (Datatype, Year) int64 34494935 ... 8...
    Low life expectancy                 (Datatype, Year) int64 34494935 ... 7...
    High migration                      (Datatype, Year) int64 34494935 ... 1...
    ...                                  ...
    Zero net migration                  (Datatype, Year) int64 34494935 ... 3...
    Young age structure                 (Datatype, Year) int64 34494935 ... 1...
    Old age structure                   (Datatype, Year) int64 34494935 ... 4...
    Replacement fertility               (Datatype, Year) int64 34494935 ... 1...
    Migration cateogry variant          (Datatype, Year) int64 34494935 ... 8...
    No long-term mortality improvement  (Datatype, Year) int64 34494935 ... 7...

In [7]:
# Write to file
ONS_population_dataset.to_netcdf("data/UK_ONS.nc")